In [1]:
import json 
import tensorflow as tf
import os
import requests

In [2]:
if not os.path.exists('/content/drive/MyDrive/MRC'):
  os.mkdir('/content/drive/MyDrive/MRC')
  os.mkdir('/content/drive/MyDrive/MRC/squad')
  os.mkdir('/content/drive/MyDrive/MRC/aaqad')

In [3]:
#uncomment if you did not download the data
#url = 'https://rajpurkar.github.io/SQuAD-explorer/dataset/'
#for file in ['train-v2.0.json', 'dev-v2.0.json']:
#    res = requests.get(f'{url}{file}')
#    # write to file
#    with open(f'/content/drive/MyDrive/MRC/squad{file}', 'wb') as f:
#        for chunk in res.iter_content(chunk_size=4):
#            f.write(chunk)

In [4]:
#uncomment if you did not download the data
#!pip install gdown
#!gdown https://drive.google.com/uc?id=1jhUmWb9eHVATqhrWKAXxSE2gqJ53-wk6 -O AAQAD.json
#!gdown https://drive.google.com/uc?id=1V5ziIZe__pGg14nH42WyMEFz444XPWf7 -O AAQAD\-train.json
#!gdown https://drive.google.com/uc?id=19nj9jiCdJlHwAfgUTJ_Z8jg1cB34yfjv -O AAQAD\-dev.json
#!gdown https://drive.google.com/uc?id=1z0XksuTwnqhiX1guxkmjYmoNA_JZ6SUN -O AAQAD\-test.json



In [5]:
with open('/content/drive/MyDrive/MRC/aaqad/AAQAD-train.json') as f:
  aaqad_train_dict= json.load(f)

In [6]:
#Dictionay(data)--ListOfArticles--Dictionary(Paragraphs)--ListOfContext--ListOfQAS
print(aaqad_train_dict.keys())
print(aaqad_train_dict['data'][0].keys())
print(aaqad_train_dict['data'][0]['paragraphs'][0])#One Context Multiple qas(key)-contains list of qas
print(aaqad_train_dict['data'][0]['paragraphs'][0]['qas'][0])
question = aaqad_train_dict['data'][0]['paragraphs'][0]['qas'][0]['question']
id =aaqad_train_dict['data'][0]['paragraphs'][0]['qas'][0]['id']
answer =aaqad_train_dict['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['text']
answer_start = aaqad_train_dict['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['answer_start']
print(f'Question: {question} it is id {id} Answer: {answer} Start Span: {answer_start}')

dict_keys(['version', 'data'])
dict_keys(['title', 'paragraphs'])
{'qas': [{'question': 'ما الذي تعتمد عليه بعض الطيور الكبيرة ذات الجناحين لمساعدتها على الارتفاع؟', 'id': 2728, 'answers': [{'text': 'تصل المسافة التي', 'answer_start': 211}], 'is_impossible': False}, {'question': 'متى تهاجر الطيور الجارحة الكبيرة؟', 'id': 2729, 'answers': [{'text': 'ليلا ونهارا،', 'answer_start': 433}], 'is_impossible': False}, {'question': 'ما مقدار وزن أجسامهم التي يفقدها طراد العسل أثناء الهجرة؟', 'id': 2730, 'answers': [{'text': 'إلى 50', 'answer_start': 245}], 'is_impossible': False}], 'context': 'هجرة الطيور هي رحلة موسمية تقوم بها أسراب من الطيور قاطعين مسافات هائلة عبر الصحاري وقمم الجبال العالية والمحيطات. تصل هذه الطيور إلى هدفها في وقت واحد يتطابق مع الوقت التي وصلت فيه في العام السابق. بعض الأنواع تصل المسافة التي تقطعها في هجرتها إلى 50 الف كيلومتر في السنة، البعض الآخر تستمر بالطيران بدون انقطاع لمدة تصل إلى 100 ساعة مع منظومة تحديد دقيقة للاتجاهات عند تلك الطيور. بعض الأنواع لها القدرة عل

In [7]:
def Read_AAQAD(path):
  contexts =[]
  answers =[]
  questions =[]
  with open(path) as f:
    aaqad_dict = json.load(f)
    for article in aaqad_dict['data']:#through each article
      for passage in article['paragraphs']:
        context = passage['context']
        for qa in passage['qas']:
          question = qa['question']
          if 'plausible_answers' in qa.keys():
            access = 'plausible_answers'
          else:
            access = 'answers'
          for answer in qa[access]:
            contexts.append(context)
            answers.append(answer)
            questions.append(question)
  return contexts,questions,answers

In [8]:
train_contexts, train_questions, train_answers = Read_AAQAD('/content/drive/MyDrive/MRC/aaqad/AAQAD-train.json')
val_contexts, val_questions, val_answers = Read_AAQAD('/content/drive/MyDrive/MRC/aaqad/AAQAD-dev.json')
test_contexts, test_questions, test_answers = Read_AAQAD('/content/drive/MyDrive/MRC/aaqad/AAQAD-test.json')

In [9]:
print(len(train_contexts))
print(len(val_contexts))
print(len(test_contexts))

12629
1926
3262


In [10]:
train_contexts[1]

'هجرة الطيور هي رحلة موسمية تقوم بها أسراب من الطيور قاطعين مسافات هائلة عبر الصحاري وقمم الجبال العالية والمحيطات. تصل هذه الطيور إلى هدفها في وقت واحد يتطابق مع الوقت التي وصلت فيه في العام السابق. بعض الأنواع تصل المسافة التي تقطعها في هجرتها إلى 50 الف كيلومتر في السنة، البعض الآخر تستمر بالطيران بدون انقطاع لمدة تصل إلى 100 ساعة مع منظومة تحديد دقيقة للاتجاهات عند تلك الطيور. بعض الأنواع لها القدرة على الطيران لمسافات طويلة، ليلا ونهارا، دون توقف. هذه القدرة هامة للغاية للتمكن من عبور الصحاري الكبرى الممتدة لالاف الكيلومترات بدون طعام أو ماء. قبل بدء رحلتهم لعبور الصحراء تقوم الطيور بأكل طعام غني بالدهون مثل حبوب الذرة.\n'

In [11]:
train_answers[1]

{'answer_start': 433, 'text': 'ليلا ونهارا،'}

In [12]:
answer = train_answers[1]['text']
start_idx =train_answers[1]['answer_start']
end_idx = start_idx + len(answer)
print(train_contexts[1][start_idx:end_idx])
print(answer)
print(answer == train_contexts[1][start_idx:end_idx])

ليلا ونهارا،
ليلا ونهارا،
True


In [13]:
def add_end_idx(answers, contexts):
  answers_idxs=[]
  s_token=-1
  e_token=-1
  cnt = 0
  for answer,context in zip(answers,contexts):
    text = answer['text']
    start_idx = answer['answer_start']
    end_idx = start_idx + len(text)
    if text != context[start_idx:end_idx]:
      if text != context[start_idx-1:end_idx-1] or text != context[start_idx-2:end_idx-2]:
        cnt = cnt+1
    answers_idxs.append([start_idx,end_idx])
  return cnt, answers_idxs

In [14]:
error_train, train_span = add_end_idx(train_answers, train_contexts)
error_val, val_span = add_end_idx(val_answers, val_contexts)
error_test, test_span = add_end_idx(test_answers, test_contexts)
total_error = error_train +error_val + error_test
ratio = total_error/(len(train_answers)+len(val_answers)+len(test_answers))
print(ratio)

0.0035920749845653028
